<a href="https://colab.research.google.com/github/PeterPirog/tf-autoencoders/blob/main/01_mnist_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
#VAE CODER
import tensorflow.keras as keras

from keras import layers
from keras import backend as K 
from keras.models import Model
import numpy as np

img_shape=(28,28,1)
batch_size=16
latent_dim =2

input_img=keras.Input(shape=img_shape)

x=layers.Conv2D(32,3,padding='same',activation='relu')(input_img)
x=layers.Conv2D(64,3,padding='same',activation='relu',strides=(2,2))(x)
x=layers.Conv2D(64,3,padding='same',activation='relu')(x)
x=layers.Conv2D(64,3,padding='same',activation='relu')(x)

shape_before_flattening=K.int_shape(x)

x=layers.Flatten()(x)
x=layers.Dense(32,activation='relu')(x)

z_mean=layers.Dense(latent_dim)(x)
z_log_var=layers.Dense(latent_dim)(x)



In [12]:

# Sampling layer
def sampling(args):
  z_mean, z_log_var=args
  epsilon=K.random_normal(shape=(K.shape(z_mean)[0],latent_dim),mean=0,stddev=1.)
  return z_mean + K.exp(z_log_var)*epsilon

z=layers.Lambda(sampling)([z_mean, z_log_var])

In [16]:
# Layer to map latent vector to image
decoder_input =layers.Input(K.int_shape(z)[1:])

x=layers.Dense(np.prod(shape_before_flattening[1:]),activation='relu')(decoder_input)
x=layers.Reshape(shape_before_flattening[1:])(x)
x=layers.Conv2DTranspose(32,3,padding='same',activation='relu',strides=(2,2))(x)
x=layers.Conv2D(1,3,padding='same',activation='sigmoid')(x)

decoder = Model(decoder_input,x)
z_decoded=decoder(z)
decoder.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 2)]               0         
                                                                 
 dense_19 (Dense)            (None, 12544)             37632     
                                                                 
 reshape_6 (Reshape)         (None, 14, 14, 64)        0         
                                                                 
 conv2d_transpose_11 (Conv2D  (None, 28, 28, 32)       18464     
 Transpose)                                                      
                                                                 
 conv2d_17 (Conv2D)          (None, 28, 28, 1)         289       
                                                                 
Total params: 56,385
Trainable params: 56,385
Non-trainable params: 0
_______________________________________________________